№1 (1 балл)
Имеется синхронная версия программы для подсчета значений факториала чисел. Используя модуль asyncio сделайте программу асинхронной. В этом случае в каком порядке завершаются задачи? С->B->A Из-за количества необходимых циклов для вычисления факториалов.

In [16]:
import asyncio

async def factorial(name, number): 
    f = 1 
    for i in range(2, number + 1): 
        print(f"Task {name}: Compute factorial({i})...")
        f *= i
        await asyncio.sleep(0) 
    print(f"Task {name}: factorial({number}) = {f}") 

async def main(): 
    await asyncio.gather(
        factorial("A", 15),
        factorial("B", 7),
        factorial("C", 4),)

if __name__ == '__main__':
    await main()

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: Compute factorial(3)...
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task A: Compute factorial(4)...
Task B: Compute factorial(4)...
Task C: Compute factorial(4)...
Task A: Compute factorial(5)...
Task B: Compute factorial(5)...
Task C: factorial(4) = 24
Task A: Compute factorial(6)...
Task B: Compute factorial(6)...
Task A: Compute factorial(7)...
Task B: Compute factorial(7)...
Task A: Compute factorial(8)...
Task B: factorial(7) = 5040
Task A: Compute factorial(9)...
Task A: Compute factorial(10)...
Task A: Compute factorial(11)...
Task A: Compute factorial(12)...
Task A: Compute factorial(13)...
Task A: Compute factorial(14)...
Task A: Compute factorial(15)...
Task A: factorial(15) = 1307674368000


№2 (2 балла)
Напишите программу, которая выводит ваш IP адрес. Есть много сервисов, которые позволяют узнать ваш ip (https://api.ipify.org, http://ip-api.com и т.д.). Но на момент запуска программы вы не знаете какой из сервисов доступен. Вместо того, чтобы опрашивать каждый из этих сервисов последовательно, запустите все запросы асинхронно и выберите первый успешный.
На экране должен быть выведен ваш ip и название сервиса, который ответил первым.

In [23]:
import asyncio
import aiohttp
from typing import Optional, Tuple

SERVICES = {
    "ipify": "https://api.ipify.org?format=json",
    "ip-api": "http://ip-api.com/json/",
}

async def fetch_ip(session: aiohttp.ClientSession, service_name: str, url: str):
    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()
            ip = data.get("ip", data.get("query"))
            return (ip, service_name)

async def get_ip():
    async with aiohttp.ClientSession() as session:
        tasks = [
            fetch_ip(session, "ipify", "https://api.ipify.org?format=json"),
            fetch_ip(session, "ip-api", "http://ip-api.com/json/") ]
        for future in asyncio.as_completed(tasks):
            result = await future
            return result

async def main():
    ip = await get_ip()
    print(ip)

if __name__ == '__main__':
    await main()

('46.181.11.34', 'ipify')


Реализуйте программу, которая эмулирует проведение собеседования. Собеседование включает 2
этапа: претенденту выдается задание, затем он идет его выполнять, представляет результат и
защищает его; потом претенденту дается 5 единиц времени отдохнуть; затем такие же действия
проводятся для второго задания, кроме отдыха, разумеется.

Напишите асинхронную функцию interviews(), которая принимает произвольное число
претендентов – кортежей вида:

(имя, время на подготовку 1 задания, время на защиту 1 задания, время подготовки 2 задания,
время на защиту второго задания)

Функция должна для каждого задания каждого претендента вывести строки:

при начале выполнения задания – <имя> started the <N> task.

при переходе к защите – <имя> moved on to the defense of the <N> task.

при окончании выполнения задания – <имя> completed the <N> task.

при начале отдыха перед вторым заданием – <имя> is resting.

Второе задание каждый претендент может получить только после выполнения первого.

Для более быстрой работы программы поделите все времена ожидания на 100.

In [20]:
import asyncio
import time

async def interview(name, task1, def1, task2, def2):
    N = 1
    print(f"{name} started {N} task.")
    await asyncio.sleep(task1 / 100)
    print(f"{name} moved on to the defense of the {N} task.")
    await asyncio.sleep(def1 / 100)
    print(f"{name} completed the {N} task.")
    N += 1
    print(f"{name} is resting.")
    await asyncio.sleep(5 / 100)
    print(f"{name} started the {N} task.")
    await asyncio.sleep(task2 / 100)
    print(f"{name} moved on to the defense of the {N} task.")
    await asyncio.sleep(def2 / 100)
    print(f"{name} completed the {N} task.")

async def interviews(*candidates):
    tasks = []
    for candidate in candidates:
        tasks.append(asyncio.create_task(interview(*candidate)))
    
    await asyncio.gather(*tasks)

if __name__ == '__main__':
    data = [
        ('Ivan', 5, 2, 7, 2),
        ('Jane', 3, 4, 5, 1),
        ('Sophia', 4, 2, 5, 1),
        ('Alexander', 2, 1, 10, 5) ]
    t0 = time.time()
    await interviews(*data)
    print(time.time() - t0)

Ivan started 1 task.
Jane started 1 task.
Sophia started 1 task.
Alexander started 1 task.
Alexander moved on to the defense of the 1 task.
Jane moved on to the defense of the 1 task.
Alexander completed the 1 task.
Alexander is resting.
Sophia moved on to the defense of the 1 task.
Ivan moved on to the defense of the 1 task.
Sophia completed the 1 task.
Sophia is resting.
Jane completed the 1 task.
Jane is resting.
Ivan completed the 1 task.
Ivan is resting.
Alexander started the 2 task.
Sophia started the 2 task.
Jane started the 2 task.
Ivan started the 2 task.
Sophia moved on to the defense of the 2 task.
Jane moved on to the defense of the 2 task.
Sophia completed the 2 task.
Alexander moved on to the defense of the 2 task.
Jane completed the 2 task.
Ivan moved on to the defense of the 2 task.
Ivan completed the 2 task.
Alexander completed the 2 task.
0.23364734649658203


Напишите асинхронную функцию sowing(), принимающую произвольное количество кортежей: (растение, время замачивания, время прорастания, время приживания после пикировки) и печатающую отчет в виде:

При начале процесса для каждого растения 0 Beginning of sowing the <растение> plant

При начале замачивания 1 Soaking of the <растение> started

После этапа замачивания 2 Soaking of the <растение> is finished

При высадке семян и установке укрытия 3 Shelter of the <растение> is supplied

При снятии укрытия 4 Shelter of the <растение> is removed

При начале пикировки 5 The <растение> has been transplanted

Растение прижилось 6 The <растение> has taken root

После окончания выращивания рассады 9 The seedlings of the <растение> are ready

Для увеличения скорости работы программы разделите все времена ожидания на 1000.

In [22]:
import asyncio
import time

async def apply_fertilizers(plant_name, secondary_checks):
    print(f"7 Application of fertilizers for {plant_name}")
    await asyncio.sleep(3 / 1000)
    print(f"7 Fertilizers for the {plant_name} have been introduced")
    secondary_checks['fertilized'] = True

async def pest_treatment(plant_name, secondary_checks):
    print(f"8 Treatment of {plant_name} from pests")
    await asyncio.sleep(5 / 1000)
    print(f"8 The {plant_name} is treated from pests")
    secondary_checks['treated_from_pest'] = True

async def grow(plant_name, soaking, growing, rooting):
    secondary_checks = {'fertilized': False, 'treated_from_pest': False}
    print(f"0 Beginning of sowing the {plant_name} plant")
    fertlz_task = asyncio.create_task(apply_fertilizers(plant_name, secondary_checks))
    pest_tr_task = asyncio.create_task(pest_treatment(plant_name, secondary_checks))
    print(f"1 Soaking of the {plant_name} started")
    await asyncio.sleep(soaking / 1000)
    print(f"2 Soaking of the {plant_name} is finished")
    print(f"3 Shelter of the {plant_name} is supplied")
    await asyncio.sleep(growing / 1000)
    print(f"4 Shelter of the {plant_name} is removed")
    print(f"5 The {plant_name} has been transplanted")
    await asyncio.sleep(rooting / 1000)
    print(f"6 The {plant_name} has taken root")
    await fertlz_task
    await pest_tr_task
    print(f"9 The seedlings of the {plant_name} are ready")

async def sowing(*plants):
    tasks = []
    for plant in plants:
        tasks.append(asyncio.create_task(grow(*plant)))

    await asyncio.gather(*tasks)

if __name__ == '__main__':
    data = [
        ('Carrot', 7, 8, 12),
        ('Cabbage', 2, 6, 10),
        ('Onion', 5, 12, 7),
        ('Tomato', 8, 7, 6) ]
    t0 = time.time()
    await sowing(*data)
    print(time.time() - t0)

0 Beginning of sowing the Carrot plant
1 Soaking of the Carrot started
0 Beginning of sowing the Cabbage plant
1 Soaking of the Cabbage started
0 Beginning of sowing the Onion plant
1 Soaking of the Onion started
0 Beginning of sowing the Tomato plant
1 Soaking of the Tomato started
7 Application of fertilizers for Carrot
8 Treatment of Carrot from pests
7 Application of fertilizers for Cabbage
8 Treatment of Cabbage from pests
7 Application of fertilizers for Onion
8 Treatment of Onion from pests
7 Application of fertilizers for Tomato
8 Treatment of Tomato from pests
2 Soaking of the Cabbage is finished
3 Shelter of the Cabbage is supplied
7 Fertilizers for the Carrot have been introduced
7 Fertilizers for the Cabbage have been introduced
7 Fertilizers for the Onion have been introduced
7 Fertilizers for the Tomato have been introduced
2 Soaking of the Onion is finished
3 Shelter of the Onion is supplied
8 The Carrot is treated from pests
8 The Cabbage is treated from pests
8 The Oni